In [1]:
import pandas as pd
import numpy as np
from IPython.display import display
from tqdm import tqdm
import pickle

from tensorflow import keras
from tensorflow.keras import models
from tensorflow.keras import callbacks

from sklearn.model_selection import GroupKFold

In [2]:
class CFG:
    dataset_dir="../input/ranzcr-clip-catheter-line-classification/"
    target_cols=['ETT - Abnormal', 'ETT - Borderline', 'ETT - Normal', 'NGT - Abnormal', 'NGT - Borderline',
       'NGT - Incompletely Imaged', 'NGT - Normal', 'CVC - Abnormal', 'CVC - Borderline', 'CVC - Normal', 'Swan Ganz Catheter Present']
    n_folds=4

In [3]:
train=pd.read_csv(f"{CFG.dataset_dir}train.csv")
y=train[CFG.target_cols]

In [4]:
npz=np.load("../input/effnet_tuned_output.npz")
features_list=[npz[uid] for uid in tqdm(train["StudyInstanceUID"])]
features=np.array(features_list)

100%|██████████| 30083/30083 [00:47<00:00, 632.10it/s]


In [5]:
scaler=pickle.load(open("./models/minmaxscaler_effnet_tuned.pickle","rb"))
X=scaler.transform(features)

In [6]:
autoencoder_dir="./models/autoencoder_tuned/"
with open(f"{autoencoder_dir}model.json","rt") as f:
    model_json=f.read()
autoencoder=models.model_from_json(model_json)
autoencoder.load_weights(f"{autoencoder_dir}ckpt")
autoencoder.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 1800)              4609800   
_________________________________________________________________
dense_1 (Dense)              (None, 100)               180100    
_________________________________________________________________
dense_2 (Dense)              (None, 1800)              181800    
_________________________________________________________________
dropout (Dropout)            (None, 1800)              0         
_________________________________________________________________
dense_3 (Dense)              (None, 2560)              4610560   
Total params: 9,582,260
Trainable params: 9,582,260
Non-trainable params: 0
_________________________________________________________________


In [7]:
layer_name="dense_1"
compressing_model=models.Model(inputs=autoencoder.input,outputs=autoencoder.get_layer(layer_name).output)

ae_pred=compressing_model.predict(X)
ae_pred_df=pd.DataFrame(ae_pred)
display(ae_pred.shape)

(30083, 100)

In [8]:
fold=train.copy()
splitter=GroupKFold(n_splits=CFG.n_folds)
for n,(train_idx,val_idx) in enumerate(splitter.split(train,groups=train["PatientID"])):
    fold.loc[val_idx,"folds"]=n
fold["folds"]=fold["folds"].astype(int)

In [12]:
def create_model():
    dense_model=keras.Sequential([
        keras.layers.Dense(len(CFG.target_cols),input_shape=(100,),activation="sigmoid")
    ])
    adam=keras.optimizers.Adam(learning_rate=1e-4)
    dense_model.compile(optimizer=adam,loss="binary_crossentropy",metrics=[keras.metrics.AUC(multi_label=True,name="auc")])
    return dense_model

In [13]:
early_stopping=callbacks.EarlyStopping(monitor="val_auc",patience=50,mode="max")
lr_reducer=callbacks.ReduceLROnPlateau(monitor="val_auc",patience=7,min_lr=1e-7,mode="max",factor=0.5,verbose=1)

In [14]:
for n in range(CFG.n_folds):
    train_idx=(fold["folds"]!=n)
    val_idx=(fold["folds"]==n)
    X_train=ae_pred_df[train_idx]
    X_val=ae_pred_df[val_idx]
    y_train=y[train_idx]
    y_val=y[val_idx]

    dense_model=create_model()
    checkpoint=callbacks.ModelCheckpoint(f"./models/effnet_tuned_dense/ckpt_{n}",monitor="val_auc",mode="max",
        save_weights_only=True,save_best_only=True)

    dense_model.fit(X_train,y_train,epochs=100,shuffle=True,validation_data=(X_val,y_val),callbacks=[early_stopping,lr_reducer,checkpoint])

    score=dense_model.evaluate(X_val,y_val,verbose=0)
    print(f"\nscore: {score}")

Epoch 1/100
706/706 [==============================] - 2s 3ms/step - loss: 0.3868 - auc: 0.5471 - val_loss: 0.2883 - val_auc: 0.6118
Epoch 2/100
706/706 [==============================] - 2s 2ms/step - loss: 0.2669 - auc: 0.6342 - val_loss: 0.2551 - val_auc: 0.6906
Epoch 3/100
706/706 [==============================] - 2s 3ms/step - loss: 0.2420 - auc: 0.7118 - val_loss: 0.2346 - val_auc: 0.7629
Epoch 4/100
706/706 [==============================] - 2s 3ms/step - loss: 0.2249 - auc: 0.7656 - val_loss: 0.2195 - val_auc: 0.7963
Epoch 5/100
706/706 [==============================] - 2s 2ms/step - loss: 0.2120 - auc: 0.7993 - val_loss: 0.2083 - val_auc: 0.8278
Epoch 6/100
706/706 [==============================] - 2s 2ms/step - loss: 0.2022 - auc: 0.8202 - val_loss: 0.1997 - val_auc: 0.8481
Epoch 7/100
688/706 [============================>.] - ETA: 0s - loss: 0.1948 - auc: 0.8423

KeyboardInterrupt: 